# Importando módulos 

In [ ]:
import obspy
from obspy.taup import TauPyModel

from multiprocessing import Pool
from obspy import read,UTCDateTime,Trace,read_inventory,read_events
from obspy.io.sac.sactrace import SACTrace
from obspy.imaging.beachball import beachball,beach
from obspy.clients.fdsn import Client
import os
import glob
import numpy as np
from collections import defaultdict
import pandas as pd
from scipy import signal
import subprocess
from sklearn import preprocessing


#para plotar as figuras
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.transforms import offset_copy
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import ListedColormap
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.cm as cm
from matplotlib.dates import YearLocator, MonthLocator, DayLocator, HourLocator, MinuteLocator, SecondLocator, DateFormatter
from matplotlib.ticker import MultipleLocator, FormatStrFormatter,FixedLocator,StrMethodFormatter
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec

from datetime import datetime,timedelta,date
from tqdm import tqdm

from shapely.geometry.polygon import LinearRing
from matplotlib.patches import Rectangle

import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter,LatitudeFormatter
import requests
import csv
import xml.etree.ElementTree as ET

# Inputs e Outputs

In [ ]:
#FOLDER_OUTPUT = '/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/OUTPUT/'
#MSEED_INPUT = "/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/DATA_GLIDER_2024/"
#LOCAL_EVENTS_TABLE = "/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/event_files/iag_catalogo_costa.csv"
#XML_FILE = '/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/XML/glider_nettab_per_dive.xml'
#QUAKEXML_FOLDER = '/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/OUTPUT/EVENTS/'
#METADATA_FILE = '/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/METADATA/df_campanha_glider.feather'
#MODEL_CAKE = '/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/EARTH_MODEL_AREA/model_ak135f_SOFAR.nd'
#RSBR_STATIONS = '/media/sysop/14f7ead0-5dcb-4557-a139-55dbb404d11a/diogoloc/dados_posdoc/Gliders_DATA/STA_COORD/estacoes_RSBR_costa.txt'

FOLDER_OUTPUT = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/'
MSEED_INPUT = "/home/sysop/dados_posdoc/GLIDER_PETROBRAS/DATA/"
XML_FILE = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/XML/glider_nettab_per_dive.xml'
METADATA_FILE = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/METADATA/df_campanha_glider.feather'
QUAKEXML_FOLDER = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/OUTPUT/EVENTS/'
MODEL_CAKE = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/EARTH_MODEL_AREA/model_ak135_SOFAR.nd'
RSBR_STATIONS = '/home/sysop/dados_posdoc/GLIDER_PETROBRAS/STA_COORD/estacoes_RSBR_costa.txt'

# Extraindo informações dos arquivos ".mseed"

In [ ]:
filenames_MSEED = sorted(glob.glob(MSEED_INPUT+'*/*/*/*/GL.*'))

In [ ]:
len(filenames_MSEED)

In [ ]:
def mseed_data_2_dataframe(i):
    subdir, filename_wav = os.path.split(i)
    filename = filename_wav.split('.mseed')[0]

    st = read(i,headonly=True)   
    #----------------------------
    #Starting Dataframe

    starttime = st[0].stats.starttime.datetime
    endtime = st[-1].stats.endtime.datetime
    
    df = pd.DataFrame([[filename],[starttime],[endtime]], index=['filename','starttime','endtime']).T
    
    #Ending Dataframe
    #----------------------------
    return df

In [ ]:
pandas_mseed_lst = []

with Pool(processes=20) as p:
    max_ = len(filenames_MSEED)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(mseed_data_2_dataframe,filenames_MSEED):
            pbar.update()
            pandas_mseed_lst.append(result)

In [ ]:
dataframe_mseed_final = pd.concat(pandas_mseed_lst, ignore_index=True)

In [ ]:
dataframe_mseed_final['starttime'] = pd.to_datetime(dataframe_mseed_final.starttime,format='%Y-%m-%d %H:%M:%S')
dataframe_mseed_final['endtime'] = pd.to_datetime(dataframe_mseed_final.endtime, format='%Y-%m-%d %H:%M:%S')
dataframe_mseed_final.sort_values('starttime')


# Extraindo informações do Inventório ".xml"

In [ ]:
inv_glider = read_inventory(XML_FILE)

In [ ]:
inv_glider

In [ ]:
df_glider_xml_lst = []
for sta in inv_glider[0]:
    for cha in sta:
        station = sta.code
        latitude = cha.latitude
        longitude = cha.longitude
        depth = cha.depth*(-1)
        starttime = cha.start_date
        endtime = cha.end_date
        df_cha = pd.DataFrame([[station],[latitude],[longitude],[depth],[starttime],[endtime]], index=['station','latitude','longitude','depth','starttime','endtime']).T
        df_glider_xml_lst.append(df_cha)

In [ ]:
dataframe_glider_xml = pd.concat(df_glider_xml_lst, ignore_index=True)

In [ ]:
dataframe_glider_xml

# RSBR SE COAST stations:

In [ ]:
RSBR_LOC = np.genfromtxt(RSBR_STATIONS,skip_header=1,delimiter=';',dtype='str')

In [ ]:
kstnm = np.genfromtxt(RSBR_STATIONS,skip_header=1,delimiter=';',dtype='str',usecols=1)
stla = np.genfromtxt(RSBR_STATIONS,skip_header=1,delimiter=';',dtype='float',usecols=2)
stlo = np.genfromtxt(RSBR_STATIONS,skip_header=1,delimiter=';',dtype='float',usecols=3)
stel = np.genfromtxt(RSBR_STATIONS,skip_header=1,delimiter=';',dtype='float',usecols=4)

In [ ]:
# Create a figure
# set up the plot and create a GeoAxes:
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(1, 1,figsize=(20,10), subplot_kw=dict(projection=proj))

# ----------------------------------------------------------------------------------------------------------
# Limit the extent of the map to a small longitude/latitude range.
latmin=-30
latmax=-20
lonmin=-50
lonmax=-40

ax.set_extent([lonmin,lonmax, latmin, latmax], crs=ccrs.Geodetic())

# ----------------------------------------------------------------------------------------------------------
# Ploting lat/lon values

h = ax.scatter(dataframe_glider_xml['longitude'],dataframe_glider_xml['latitude'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_glider_xml['starttime']]),marker='.',alpha=0.5,cmap='plasma',s=20,transform=proj)
ax.scatter(stlo,stla,c='k',marker='^',ec='w',s=70,transform=proj)

# Use the cartopy interface to create a matplotlib transform object
# for the Geodetic coordinate system. We will use this along with
# matplotlib's offset_copy function to define a coordinate system which
# translates the text by 25 pixels to the left.
geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
text_transform = offset_copy(geodetic_transform, units='dots', x=0,y=10)

for lo,la,name in zip(stlo,stla,kstnm):
    if lo < lonmax and lo > lonmin and la > latmin and la < latmax:
        ax.text(lo,la,s=name, verticalalignment='center', horizontalalignment='center',alpha=0.5,transform=text_transform)
  
# ----------------------------------------------------------------------------------------------------------
# Adding background map 
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE,linewidth=0.3)
ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')

# ----------------------------------------------------------------------------------------------------------
# Adding colorbar
divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size="1%", pad=0.6, axes_class=plt.Axes)

fig.add_axes(ax_cb)
cb = plt.colorbar(h, cax=ax_cb)
cb.ax.yaxis.set_major_formatter(mdates.DateFormatter('%d/%m/%y'))

#####################################################
# Adding inset axes LEFT
#####################################################

# ----------------------------------------------------------------------------------------------------------
# Adding global location map
# Adding Geoaxes
ax_map = plt.axes([0.2, 0.8, 0.2, 0.2], projection=ccrs.Orthographic(central_latitude=(latmin + latmax)/2,central_longitude=(lonmin + lonmax) / 2))
ax_map.set_global()

# Adding background map 
ax_map.add_feature(cfeature.LAND)
ax_map.add_feature(cfeature.OCEAN)
ax_map.add_feature(cfeature.COASTLINE)

# Adding red rectangle position
nvert = 100
lons = np.r_[np.linspace(lonmin, lonmin, nvert),
             np.linspace(lonmin, lonmax, nvert),
             np.linspace(lonmax, lonmax, nvert)].tolist()
lats = np.r_[np.linspace(latmin, latmax, nvert),
             np.linspace(latmax, latmax, nvert),
             np.linspace(latmax, latmin, nvert)].tolist()

ring = LinearRing(list(zip(lons, lats)))
ax_map.add_geometries([ring], ccrs.PlateCarree(),
                   facecolor='none', edgecolor='red', linewidth=0.75)


# Buscando eventos com mecanismos focais através do pyGCMT

In [ ]:
cat = obspy.read_events(QUAKEXML_FOLDER+'CMTSOLUTIONS')
cat

In [ ]:
cat.plot()

# PROCURANDO EVENTOS EM FUNÇÃO DO CATÁLOGO 

In [ ]:
def classic_sta_lta_py(a, nsta, nlta):
    """
    Computes the standard STA/LTA from a given input array a. The length of
    the STA is given by nsta in samples, respectively is the length of the
    LTA given by nlta in samples. Written in Python by Obspy.
    """
    m = len(a)
    #
    # compute the short time average (STA) and long time average (LTA)
    sta = np.zeros(m, dtype=np.float64)
    lta = np.zeros(m, dtype=np.float64)

    for i in range(m):
        sta[i] = np.mean(a[i:int(i+nsta)]**2)
        lta[i] = np.mean(a[i:int(i+nlta)]**2)

    # Pad zeros
    sta[:nlta - 1] = 0

    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny

    return sta / lta

In [ ]:
# Função de filtro para a condição especificada
def condition(lst):
    return not lst[0] and not lst[1] and not lst[2] and (lst[3] or lst[4])
           

In [ ]:

def event_search(inpt,CUT_BEFORE=-30,CUT_AFTER=180,FREQ_CFT=3,STA_short=2,LTA_short=40,THRON=5,THROFF=2,SNR_MIN=2,event_cut_sta_lta_band=20,nl_start=2,nl_end=9,sl_start=1,sl_end=6,model_earth='ak135'):
        '''
        Function to search and plot events
        
        CUT_BEFORE:
        CUT_AFTER:,
        FREQ_CFT:,
        STA_short:,
        LTA_short,
        THRON:,
        event_cut_sta_lta_band:,
        nl_start:, 
        nl_end:,
        sl_start:,
        sl_end:,
        model_earth:
        ''' 
    #try:
        ev_file = inpt[0]
        seismic_phase = inpt[1]
    
        # =========================================
        # CODE
        
        # Definindo globalmente o tamanho da fonte
        mpl.rcParams.update({'font.size': 14})
      
        ev = ev_file
        
        ev_time = ev.origins[0].time
        ev_lat = ev.origins[0].latitude
        ev_long = ev.origins[0].longitude
        ev_depth = ev.origins[0].depth/1000

        file_mseed_info = dataframe_mseed_final[(dataframe_mseed_final.starttime <= ev_time.datetime) & (dataframe_mseed_final.endtime >= ev_time.datetime)]
        if len(file_mseed_info['filename'].tolist()) > 0:
            name_glider = file_mseed_info['filename'].tolist()[0].split('.')[1]

            # Selecionando no inventório dos glider a localização da estação via ARQUIVO XML:
            inv_sel = inv_glider.select(channel="*H", station=name_glider)
            sta_sel = inv_sel[0][0]
            for cha in sta_sel:
                if (cha.start_date <= ev_time) & (cha.end_date >= ev_time):
                    
                    #Approximation 
                    lat_glider = cha.latitude
                    lon_glider = cha.longitude
                    dep_glider = -abs(cha.depth)
                    time_glider = cha.start_date
    
                    #Calculating distance, azimuth and backazimuth
                    dist,az,baz = obspy.geodetics.gps2dist_azimuth(ev_lat,ev_long,lat_glider,lon_glider)
                    gcarc = obspy.geodetics.kilometer2degrees(dist/1000)
    
                    # Calculating arrival time by CAKE (https://pyrocko.org/docs/current/apps/cake/index.html)
                    # Cake is a command line tool contained in Pyrocko, which can solve some classical seismic ray theory problems for 1D layered earth models (layer cake models).
    
                    command = 'cake arrivals --model='+MODEL_CAKE+' --sdepth='+str(ev_depth)+' --rdepth='+str(np.abs(dep_glider)/1000)+' --distances='+str(gcarc)+' --degrees --classic='+seismic_phase
                    proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True,text=True)
                    (out, err) = proc.communicate()
                    results = out.splitlines()
                        
                    if len(results) > 3:
                        event = UTCDateTime(ev_time)+float(results[3].split(' ')[2])
    
                        P_label = seismic_phase
                        
                        #########################################################################################################################################################
                        #STREAM 
    
                        file_mseed_event = dataframe_mseed_final[(dataframe_mseed_final.starttime <= event.datetime) & (dataframe_mseed_final.endtime >= event.datetime)]
    
                        if len(file_mseed_event) > 0:
                            ev_file_mseed = file_mseed_event['filename'].values[0]
                            file_mseed_ev = sorted(glob.glob(MSEED_INPUT+'*/*/*/*/'+ev_file_mseed+'*'))[0]
    
                            stream_mseed = read(file_mseed_ev)
                            stream_mseed = stream_mseed.trim(event+CUT_BEFORE,event+CUT_AFTER)
                            
                            stream_mseed_test = stream_mseed.copy()
                            if len(stream_mseed_test.trim(event-event_cut_sta_lta_band,event+event_cut_sta_lta_band)) > 0:
    
                                # importing the true parameter values of the glider (lon,lat,dep)
                                metadata = pd.read_feather(METADATA_FILE)
                                
                                metadata_sel = metadata[metadata.station_nettab == name_glider]
    
                                lat_glider_meta = [i for i in metadata_sel["lat"].values[0]]
                                lon_glider_meta = [i for i in metadata_sel["lon"].values[0]]
                                dep_glider_meta = [i*-1 for i in metadata_sel["dep"].values[0]]
                                time_glider_meta = [pd.to_datetime(i) for i in metadata_sel["time"].values[0]]
                                name_glider_df = pd.DataFrame([lat_glider_meta,lon_glider_meta,dep_glider_meta,time_glider_meta], index=['lat_glider','lon_glider','dep_glider','time_glider']).T
                
                                name_glider_df_sel = name_glider_df[(name_glider_df.time_glider >= (event+CUT_BEFORE).datetime) & (name_glider_df.time_glider <= (event+CUT_AFTER).datetime)]
    
                                if len(name_glider_df_sel) > 0:
                                    lat_glider = name_glider_df_sel.lat_glider.values[0]
                                    lon_glider = name_glider_df_sel.lon_glider.values[0]
                                    dep_glider = name_glider_df_sel.dep_glider.values[0]
                                    time_glider = name_glider_df_sel.time_glider.values[0]
                                    
                                lat_event_info = dataframe_glider_xml[dataframe_glider_xml['station'].str.contains(name_glider)]['latitude'].values
                                lon_event_info = dataframe_glider_xml[dataframe_glider_xml['station'].str.contains(name_glider)]['longitude'].values        
                                depth_event_info = dataframe_glider_xml[dataframe_glider_xml['station'].str.contains(name_glider)]['depth'].values       
                                time_event_info = dataframe_glider_xml[dataframe_glider_xml['station'].str.contains(name_glider)]['starttime'].values        
    
                                #########################################################################################################################################################
                                #Saving 
                                headerHHH = {
                                            'kstnm': name_glider, 'kcmpnm': 'HHH','knetwk':'GL',
                                            'stla': float(lat_glider), 'stlo': float(lon_glider),'stel': float(dep_glider),
                                            'evdp': float(ev_depth), 'evla': float(ev_lat), 'evlo': float(ev_long), 'mag': float(ev.magnitudes[0].mag),
                                            'nzhour': int(event.hour),'nzjday': int(event.julday), 'nzmin': int(event.minute), 'nzmsec': int('{:03}'.format(event.microsecond)[:3]),'nzsec': int(event.second),'nzyear': int(event.year),
                                            'cmpaz': 0.0, 'cmpinc': 0.0, 'dist': float(dist/1000), 'gcarc': float(gcarc), 'az': float(az), 'baz': float(baz),
                                            't1':float(results[3].split(' ')[2]),'delta':stream_mseed[0].stats.delta
                                            }
    
                                sacHHH = SACTrace(data=stream_mseed[0].data, **headerHHH)
    
                                #Creating Event Directory
                                output_sac_data = FOLDER_OUTPUT+'EVENTS_GLIDERS/'+'GL'+'/'+name_glider+'/'+'{:04}'.format(ev_time.year)+'/'+'{:03}'.format(ev_time.julday)+'/'+'{:04}'.format(ev_time.year)+'.'+'{:03}'.format(ev_time.julday)+'.'+'{:02}'.format(ev_time.hour)+'.'+'{:02}'.format(ev_time.minute)+'.'+'{:02}'.format(ev_time.second)+'.'+'{:02}'.format(ev_time.microsecond)[:3]+'/'
                                os.makedirs(output_sac_data,exist_ok=True)
                                sacHHH.write(output_sac_data+'GL'+'.'+name_glider+'.'+'{:04}'.format(ev_time.year)+'.'+'{:03}'.format(ev_time.julday)+'.'+'{:02}'.format(ev_time.hour)+'.'+'{:02}'.format(ev_time.minute)+'.'+'{:02}'.format(ev_time.second)+'.'+'{:02}'.format(ev_time.microsecond)[:3]+'.H')
                                        
                                # ----------------------------------------------------------------------------------------------------------
                                # set up the plot and create a GeoAxes:
                                # ----------------------------------------------------------------------------------------------------------
    
                                proj = ccrs.PlateCarree()
    
                                fig = plt.figure(constrained_layout=True,figsize=(30,20))
                                
                                spec2 = gridspec.GridSpec(ncols=3, nrows=9, figure=fig)
                                ax = fig.add_subplot(spec2[0:3,:], projection=proj)
                                ax1 = fig.add_subplot(spec2[3:6,0])
                                ax2 = fig.add_subplot(spec2[6:9,0],sharex=ax1)
    
                                # ----------------------------------------------------------------------------------------------------------
                                # Limit the extent of the map to a small longitude/latitude range.
                                # ----------------------------------------------------------------------------------------------------------
    
                                latmin=-28.5
                                latmax=-22.5
                                lonmin=-50
                                lonmax=-40
    
                                ax.set_extent([lonmin,lonmax, latmin, latmax], crs=ccrs.Geodetic())
    
                                # ----------------------------------------------------------------------------------------------------------
                                # Ploting lat/lon values
                                # ----------------------------------------------------------------------------------------------------------
    
                                h = ax.scatter(dataframe_glider_xml['longitude'],dataframe_glider_xml['latitude'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_glider_xml['starttime']]),marker='o',alpha=0.5,cmap='plasma',s=25,transform=proj)
                                ax.scatter(lon_glider, lat_glider, color='r',marker='^',s=100,ec='k',transform=proj)
    
                                ax.scatter(stlo,stla,c='k',marker='^',ec='w',s=70,transform=proj)
                                
                                # Use the cartopy interface to create a matplotlib transform object
                                # for the Geodetic coordinate system. We will use this along with
                                # matplotlib's offset_copy function to define a coordinate system which
                                # translates the text by 25 pixels to the left.
                                geodetic_transform = ccrs.Geodetic()._as_mpl_transform(ax)
                                text_transform = offset_copy(geodetic_transform, units='dots', x=0,y=10)
                                
                                # Add text 25 pixels to the left of the volcano.
                                for lo,la,name in zip(stlo,stla,kstnm):
                                    if lo < lonmax and lo > lonmin and la > latmin and la < latmax:
                                        ax.text(lo,la,s=name, verticalalignment='center', horizontalalignment='center',alpha=0.5,transform=text_transform)
                                
                                # ----------------------------------------------------------------------------------------------------------
                                # Adding background map 
                                # ----------------------------------------------------------------------------------------------------------
                                
                                ax.add_feature(cfeature.LAND)
                                ax.add_feature(cfeature.OCEAN)
                                ax.add_feature(cfeature.COASTLINE,linewidth=0.3)
                                ax.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)
                                ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='gray', alpha=0.5, linestyle='--')
    
                                #Title
                                ax.set_title(ev.origins[0].time.strftime('%Y-%m-%dT%H:%M:%S')+'\n Dist: '+str(round(gcarc))+'$^\circ$'+' Dep: '+str(round(ev_depth))+' km'+'\n Mag: '+str(ev.magnitudes[0].mag)+' '+ev.magnitudes[0].magnitude_type+'\n ',y=1.02,fontsize=20)
    
                                # ----------------------------------------------------------------------------------------------------------
                                # Adding colorbar
                                # ----------------------------------------------------------------------------------------------------------
    
                                divider = make_axes_locatable(ax)
                                ax_cb = divider.new_horizontal(size="1%", pad=0.6, axes_class=plt.Axes)
    
                                fig.add_axes(ax_cb)
                                cb = plt.colorbar(h, cax=ax_cb)
                                cb.ax.yaxis.set_major_formatter(mdates.DateFormatter('%b-%Y'))
    
                                #####################################################
                                # Adding inset LAT LON
                                #####################################################
    
                                axins = plt.axes([0.8, 0.70, 0.15, 0.15],projection=proj)
    
                                axins.scatter(lon_event_info,lat_event_info,c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in time_event_info]),marker='o',alpha=0.3,cmap='plasma',s=25,transform=proj,vmax=max([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_glider_xml['starttime']]),vmin=min([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_glider_xml['starttime']]))
                                axins.scatter(lon_glider, lat_glider, color='r',marker='^',s=100,ec='k',transform=proj)
    
                                axins.add_feature(cfeature.LAND)
                                axins.add_feature(cfeature.OCEAN)
                                axins.add_feature(cfeature.COASTLINE,linewidth=0.3)
                                axins.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)
    
                                # ----------------------------------------------------------------------------------------------------------
                                # Adding grid
                                # ----------------------------------------------------------------------------------------------------------
    
                                axins.gridlines(crs=ccrs.PlateCarree(), draw_labels=["bottom", "left", "right"],linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
                                axins.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=True, labeltop=False, labelleft=True, labelright=True)
    
                                # subregion of the original image
                                x1 = np.min(lon_event_info)-abs(np.min(lon_event_info)/50)
                                x2 = np.max(lon_event_info)+abs(np.max(lon_event_info)/50)
                                y1 = np.min(lat_event_info)-abs(np.min(lat_event_info)/50)
                                y2 = np.max(lat_event_info)+abs(np.max(lat_event_info)/50)
                                axins.set_xlim(x1, x2)
                                axins.set_ylim(y1, y2)
                                ax.indicate_inset_zoom(axins, edgecolor="black")
    
                                #####################################################
                                # Adding inset DEPTH LON
                                #####################################################
    
                                ax_histx = plt.axes([0.8, 0.86, 0.15, 0.05],sharex=axins,facecolor='lightsteelblue')
                                ax_histx.scatter(lon_glider,dep_glider, c=np.array(mdates.date2num(time_glider)),marker='o',vmax=max([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_glider_xml['starttime']]),vmin=min([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_glider_xml['starttime']]),alpha=0.5,cmap='plasma',s=100)
                                ax_histx.grid(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
                                ax_histx.set_title("Depth: "+str(round(dep_glider))+' m',fontsize=13)
                                ax_histx.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=False, labeltop=False, labelleft=True, labelright=True)
                                ax_histx.set_ylim(-1000,0)
                                ax_histx.yaxis.set_major_formatter('{x} m')
                                ax_histx.yaxis.set_major_locator(MultipleLocator(500))
                                ax_histx.yaxis.set_minor_locator(MultipleLocator(100))
    
                                #####################################################
                                # Adding global location map
                                #####################################################
        
                                # Adding Geoaxes
                                ax_map = plt.axes([0.25, 0.85, 0.13, 0.13], projection=ccrs.Orthographic(central_latitude=(latmin + latmax)/2,central_longitude=(lonmin + lonmax) / 2))
                                ax_map.set_global()
    
                                # Adding background map 
                                ax_map.add_feature(cfeature.LAND)
                                ax_map.add_feature(cfeature.OCEAN)
                                ax_map.add_feature(cfeature.COASTLINE)
        
                                # Adding red rectangle position
                                nvert = 100
                                lons = np.r_[np.linspace(lonmin, lonmin, nvert),
                                                     np.linspace(lonmin, lonmax, nvert),
                                                     np.linspace(lonmax, lonmax, nvert)].tolist()
                                lats = np.r_[np.linspace(latmin, latmax, nvert),
                                                     np.linspace(latmax, latmax, nvert),
                                                     np.linspace(latmax, latmin, nvert)].tolist()
    
                                ring = LinearRing(list(zip(lons, lats)))
                                ax_map.add_geometries([ring], ccrs.PlateCarree(),facecolor='none', edgecolor='red', linewidth=0.75)
    
                                ax_map.scatter(ev_long,ev_lat,color="y",marker='*',s=200,ec='k',transform=ccrs.PlateCarree())
                                ax_map.scatter(np.mean(lon_event_info),np.mean(lat_event_info),color="r",marker='^',s=50,transform=ccrs.PlateCarree())
                                ax_map.plot([np.mean(lon_event_info), ev_long], [np.mean(lat_event_info), ev_lat], c='gray',ls='-',lw=2, transform=ccrs.Geodetic())
    
                                # ==========================================================================================================
                                # Adding stream plot
                                # ==========================================================================================================
    
                                ax1_HF = ax1.twinx()
    
                                st_LF = stream_mseed.copy()
                                st_HF = stream_mseed.copy()
    
                                for tr in st_HF:
                                    tr.detrend("linear")
                                    tr.taper(max_percentage=0.05, type='cosine')
                                    tr.filter('highpass', freq=FREQ_CFT, corners=4, zerophase=True)
                                    line_HF, = ax1_HF.plot(tr.times('matplotlib'), tr.data,'grey', lw=2,alpha=0.5,label='> '+str(FREQ_CFT)+' Hz')
                                ax1_HF.legend(handles=[line_HF],loc='lower right', fontsize=10, edgecolor='grey', labelcolor='grey')   
    
                                for tr in st_LF:
                                    tr.detrend("linear")
                                    tr.taper(max_percentage=0.05, type='cosine')
                                    tr.filter('lowpass', freq=FREQ_CFT, corners=4, zerophase=True)
                                    line_LF, = ax1.plot(tr.times('matplotlib'), tr.data,'k', lw=1, label='< '+str(FREQ_CFT)+' Hz')
                                ax1.legend(handles=[line_LF],loc='lower left', fontsize=10, edgecolor='k', labelcolor='k')
    
                                ax1.text(x=event.matplotlib_date,y=np.max(st_LF[0].data)+abs(np.max(st_LF[0].data)/50),s=P_label,ha="center", va="center",bbox=dict(boxstyle="round",ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8)))
                                ax1.axvline(x=event.matplotlib_date,ymin=0, ymax=1, color='r',linestyle='--')
    
                                ax1.set_ylabel('Amplitude [counts]')
                                ax1.set_ylim(-1.5*abs(np.max(st_LF[0].data)),1.5*abs(np.max(st_LF[0].data)))
                                ax1.set_xlim(st_LF[0].times('matplotlib')[0],st_LF[0].times('matplotlib')[-1])
                                locator = mdates.AutoDateLocator(minticks=9, maxticks=14)
                                formatter = mdates.ConciseDateFormatter(locator)
                                ax1.xaxis.set_major_locator(locator)
                                ax1.xaxis.set_major_formatter(formatter)
                                ax1.set_title('Station: '+name_glider)
    
                                ax1_HF.set_ylim(-1.5*abs(np.max(st_HF[0].data)),1.5*abs(np.max(st_HF[0].data)))
                                ax1_HF.set_xlim(st_HF[0].times('matplotlib')[0],st_HF[0].times('matplotlib')[-1])
                            
                                ax1.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=False,labelbottom=False, labeltop=False, labelleft=True, labelright=False)
                                ax1_HF.tick_params(axis='both',which="both",bottom=False, top=False, left=False, right=True,labelbottom=False, labeltop=False, labelleft=False, labelright=True)
 
                                ax1.spines['top'].set_linewidth(2)
                                ax1.spines['right'].set_linewidth(2)
                                ax1.spines['bottom'].set_linewidth(2)
                                ax1.spines['left'].set_linewidth(2)
                                ax1.xaxis.set_tick_params(width=2)
                                ax1.yaxis.set_tick_params(width=2)      
                                # ==========================================================================================================
                                # Adding spectrogram plot
                                # ==========================================================================================================
    
                                fs = stream_mseed[0].stats.sampling_rate  # Frequência de amostragem (100 amostras por segundo)
                                signal_data = stream_mseed[0].data
    
                                # Spectrogram
                                frequencies, times, Sxx = signal.spectrogram(x=signal_data, fs=fs,nperseg=128,scaling='spectrum')
    
                                # Plot spectrogram
                                im = ax2.imshow(10 * np.log10(Sxx),extent=[st_HF[0].times('matplotlib')[0],st_HF[0].times('matplotlib')[-1], 0, fs/2],aspect='auto',origin='lower',cmap='magma',interpolation='kaiser')
                                ax2.xaxis.set_major_locator(locator)
                                ax2.xaxis.set_major_formatter(formatter)
                                
                                ax2.spines['top'].set_linewidth(2)
                                ax2.spines['right'].set_linewidth(2)
                                ax2.spines['bottom'].set_linewidth(2)
                                ax2.spines['left'].set_linewidth(2)
                                ax2.xaxis.set_tick_params(width=2)
                                ax2.yaxis.set_tick_params(width=2)  
        
                                # ----------------------------------------------------------------------------------------------------------
                                
                                ax2.text(x=event.matplotlib_date,y=45,s=P_label,ha="center", va="center",bbox=dict(boxstyle="round",ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8)))
                                ax2.axvline(x=event.matplotlib_date,ymin=0, ymax=1, color='w',linestyle='--')
                                ax2.set_ylabel('Frequency [Hz]')
    
                                # ----------------------------------------------------------------------------------------------------------
                                # colorbar
                                # ----------------------------------------------------------------------------------------------------------
    
                                ax_cbar = inset_axes(ax2,
                                                            width="20%",  # width = 15% of parent_bbox width
                                                            height="3%",  # height : 3%
                                                            loc='upper right',
                                                            bbox_to_anchor=(-0.1, -0.1, 1, 1),
                                                            bbox_transform=ax2.transAxes,
                                                            borderpad=0
                                                            )
                                
                                cbar = fig.colorbar(im, cax=ax_cbar, orientation="horizontal",ticklocation='top')
                                cbar.outline.set_edgecolor('white')
                                cbar.ax.xaxis.set_tick_params(color='white')
                                plt.setp(plt.getp(cbar.ax, 'xticklabels'), color='white')
                                cbar.outline.set_linewidth(2)
    
                                # ==========================================================================================================
                                # focal mechanism
                                # ==========================================================================================================
    
                                newax = fig.add_axes([0.05, 0.53, 0.25,  0.25])
    
                                moment_tensor = ev.focal_mechanisms[0].moment_tensor
                                mrr = moment_tensor.tensor.m_rr
                                mtt = moment_tensor.tensor.m_tt
                                mpp = moment_tensor.tensor.m_pp
                                mrt = moment_tensor.tensor.m_rt
                                mrp = moment_tensor.tensor.m_rp
                                mtp = moment_tensor.tensor.m_tp
                                mt = [mrr, mtt, mpp, mrt, mrp, mtp]
    
                                bball = beach(fm=mt, xy=(0, 0.5),size=200, width=0.75, facecolor='b')
    
                                # Configura os eixos e adiciona a coleção
                                newax.add_collection(bball)
                                newax.set_xlim(-1, 1)
                                newax.set_ylim(-1, 1)
                                newax.set_aspect('equal')
                                newax.axis('off')
                                newax.set_title('Focal mechanism')
    
                                # ==========================================================================================================
                                # ray paths
                                # ==========================================================================================================
                                
                                model = TauPyModel(model=model_earth)
                                arrivals_ray_path = model.get_ray_paths(source_depth_in_km=ev_depth, distance_in_degree=gcarc, phase_list=[P_label])
                                ax_raypath = fig.add_axes([0.11, 0.81, 0.13,  0.13], projection='polar')
                                arrivals_ray_path.plot_rays(ax=ax_raypath)
                                ax_raypath.set_title('Ray path',y=1.08)
    
                                #########################################################################################################################################################
                                # Figure
                                #########################################################################################################################################################
                                
                                filter_class = [['Raw Data'],[25,50],[10,25],[5,10],[2,5],[0.7,2]]
    
                                mins2 = MultipleLocator(5)
                                mins1 = MultipleLocator(1)
    
                                check_SNR_STA_LTA = []                             
                                srn_per_band_lst = []                             
                                sta_lta_max_per_band_lst = []                             
                                noise_per_band_lst = []

                                for idx,freqs in enumerate(filter_class):
    
                                    axs1 = fig.add_subplot(spec2[3+idx,1]) 
                                
                                    axs2 = fig.add_subplot(spec2[3+idx,2])
                                    
                                    st_full = stream_mseed.copy()
                                    st_full.trim(event-event_cut_sta_lta_band,event+event_cut_sta_lta_band)
                                    
                                    time_x_axis = st_full[0].times() - event_cut_sta_lta_band
                                    st_full[0].data = preprocessing.normalize([st_full[0].data])[0]
                                    if idx == 0:
                                        
                                        st_full.detrend("linear")
                                        st_full.taper(max_percentage=0.05, type='cosine')
                                        
                                        line_full, = axs1.plot(time_x_axis, st_full[0].data,'k', lw=0.5,label=freqs[0])
                                        axs1.legend(handles=[line_full],loc='upper left', fontsize=10, edgecolor='k', labelcolor='k')   
                                        axs1.axvline(x=0,ymin=0, ymax=1, color='k',lw=0.5,linestyle='--')
                                        axs1.set_xlim(-event_cut_sta_lta_band,event_cut_sta_lta_band)
                                        axs1.xaxis.set_major_locator(mins2)
                                        axs1.xaxis.set_minor_locator(mins1)
                                        axs1.set_title('Time relative to theoretical arrival (s)')
    
                                        # ----------------------------------------------------------------------------------------------------------
                                        # SNR P-wave
                                        # ----------------------------------------------------------------------------------------------------------
    
                                        # Retrieving the P-wave normalized amplitude between -x and +x seconds
                                        amp_P = st_full[0].data[(event_cut_sta_lta_band-sl_start)*100:(event_cut_sta_lta_band+sl_end)*100]
                                        amp_P_time = st_full[0].times()[(event_cut_sta_lta_band-sl_start)*100:(event_cut_sta_lta_band+sl_end)*100] - event_cut_sta_lta_band
           
                                        # Retrieving the noise normalized amplitude between -x and +x seconds
                                        noise_amp = st_full[0].data[(event_cut_sta_lta_band-nl_end)*100:(event_cut_sta_lta_band-nl_start)*100]
                                        noise_time = st_full[0].times()[(event_cut_sta_lta_band-nl_end)*100:(event_cut_sta_lta_band-nl_start)*100] - event_cut_sta_lta_band
    
                                        # Estime a SNR
                                        powS = np.mean(np.abs(amp_P))
                                        powN = np.mean(np.abs(noise_amp))
                                        
                                        snr = powS/powN
                                        
                                        # Calcular a potência média do ruído
                                        potencia_ruido = np.mean(noise_amp**2)
                                        
                                        # Calcular o nível de ruído em dB
                                        nivel_ruido_db = 10 * np.log10(potencia_ruido)
            
                                        axs1.text(x=0.5,y=0.9,s='SRN: '+str(round(snr,2)),ha="center", va="center",horizontalalignment='center',verticalalignment='center', transform=axs1.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        axs1.plot(amp_P_time, amp_P,'darkred', alpha=0.85,lw=1)
                                        axs1.plot(noise_time, noise_amp,'darkblue', alpha=0.85,lw=1)                                       
                                        axs1.text(x=0.005,y=0.03,s='Noise: '+str(round(nivel_ruido_db,2))+' dB',ha="left", va="bottom",transform=axs1.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        
                                        axs1.spines['top'].set_linewidth(2)
                                        axs1.spines['right'].set_linewidth(2)
                                        axs1.spines['bottom'].set_linewidth(2)
                                        axs1.spines['left'].set_linewidth(2)
                                        axs1.xaxis.set_tick_params(width=2)
                                        axs1.yaxis.set_tick_params(width=2)                                
                                        # ----------------------------------------------------------------------------------------------------------                                   
                                        # STA/LTA
                                        # ----------------------------------------------------------------------------------------------------------
    
                                        st_cft = st_full.copy()
                
                                        for tr in st_cft:
                                            tr.detrend("linear")
                                            tr.taper(max_percentage=0.05, type='cosine')
                                            tr.filter('lowpass', freq=FREQ_CFT, corners=4, zerophase=True)
        
                                        cft = classic_sta_lta_py(st_full[0].data, int((STA_short/2)*st_full[0].stats.sampling_rate), int((LTA_short/2)*st_full[0].stats.sampling_rate))
                                        
                                        sta_lta_max_amp = np.max(cft)
                                        sta_lta_max_time = time_x_axis[np.argmax(cft)]
                                        axs2.scatter(sta_lta_max_time,sta_lta_max_amp, edgecolors='r',facecolors='None',marker='o',linewidths=1)
                                        
                                        axs2.text(x=0.5,y=0.9,s='max: '+str(round(sta_lta_max_amp,1)),ha="center", va="center",horizontalalignment='center',verticalalignment='center', transform=axs2.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        axs2.text(x=0.99,y=0.95,s='THRON: '+str(THRON)+'\n'+'THROFF: '+str(THROFF),ha="right", va="top",transform=axs2.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        axs2.text(x=0.01,y=0.03,s='STA: '+str(STA_short)+'\n'+'LTA: '+str(LTA_short),ha="left", va="bottom",transform=axs2.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                           
                                        line_full, = axs2.plot(time_x_axis,cft, 'k',label=freqs[0])
                                        axs2.set_ylim(0, 10)
                                        axs2.legend(handles=[line_full],loc='upper left', fontsize=10, edgecolor='k', labelcolor='k')   
                                        axs2.axvline(x=0,ymin=0, ymax=1, color='k',lw=0.5,linestyle='--')
                                        axs2.axhline(xmin=0,xmax=1, y=THRON, color='darkorange',lw=0.5,linestyle='--')
                                        axs2.axhline(xmin=0,xmax=1, y=THROFF, color='tan',lw=0.5,linestyle='--')
                                        
                                        axs2.set_xlim(-event_cut_sta_lta_band,event_cut_sta_lta_band)
                                        axs2.xaxis.set_major_locator(mins2)
                                        axs2.xaxis.set_minor_locator(mins1)
                                        axs2.set_title('Time relative to theoretical arrival (s)')

                                        axs2.spines['top'].set_linewidth(2)
                                        axs2.spines['right'].set_linewidth(2)
                                        axs2.spines['bottom'].set_linewidth(2)
                                        axs2.spines['left'].set_linewidth(2)
                                        axs2.xaxis.set_tick_params(width=2)
                                        axs2.yaxis.set_tick_params(width=2)  
    
                                    if idx > 0:
                                        
                                        st_full.filter('bandpass', freqmin=freqs[0],freqmax=freqs[1], corners=4, zerophase=False)
                                        st_full.taper(max_percentage=0.05, type='cosine')                                    
                                    
                                        line_full, = axs1.plot(time_x_axis,st_full[0].data,'k', lw=0.5,label=str(freqs[0])+'-'+str(freqs[1])+' Hz')
                                        axs1.legend(handles=[line_full],loc='upper left', fontsize=10, edgecolor='k', labelcolor='k')   
                                        axs1.axvline(x=0,ymin=0, ymax=1, color='k',lw=0.5,linestyle='--')
                                        axs1.set_xlim(-event_cut_sta_lta_band,event_cut_sta_lta_band)
                                        axs1.xaxis.set_major_locator(mins2)
                                        axs1.xaxis.set_minor_locator(mins1)
    
                                        # ----------------------------------------------------------------------------------------------------------
                                        # SNR P-wave
                                        # ----------------------------------------------------------------------------------------------------------
            
                                        # Retrieving the P-wave normalized amplitude between -x and +x seconds
                                        amp_P = st_full[0].data[(event_cut_sta_lta_band-sl_start)*100:(event_cut_sta_lta_band+sl_end)*100]
                                        amp_P_time = st_full[0].times()[(event_cut_sta_lta_band-sl_start)*100:(event_cut_sta_lta_band+sl_end)*100] - event_cut_sta_lta_band
           
                                        # Retrieving the noise normalized amplitude between -x and +x seconds
                                        noise_amp = st_full[0].data[(event_cut_sta_lta_band-nl_end)*100:(event_cut_sta_lta_band-nl_start)*100]
                                        noise_time = st_full[0].times()[(event_cut_sta_lta_band-nl_end)*100:(event_cut_sta_lta_band-nl_start)*100] - event_cut_sta_lta_band
    
                                        # Estime a SNR
                                        powS = np.mean(np.abs(amp_P))
                                        powN = np.mean(np.abs(noise_amp))
                                        
                                        snr = powS/powN
                                        
                                        # Calcular a potência média do ruído
                                        potencia_ruido = np.mean(noise_amp**2)
                                        
                                        # Calcular o nível de ruído em dB
                                        nivel_ruido_db = 10 * np.log10(potencia_ruido)
    
                                        
                                        # Plotting the SNR
                                        axs1.text(x=0.5,y=0.9,s='SRN: '+str(round(snr,2)),ha="center", va="center", transform=axs1.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        axs1.plot(amp_P_time, amp_P,'darkred', alpha=0.85,lw=1)
                                        axs1.plot(noise_time, noise_amp,'darkblue', alpha=0.85, lw=1)                                 
                                        axs1.text(x=0.005,y=0.03,s='Noise: '+str(round(nivel_ruido_db,2))+' dB',ha="left", va="bottom",transform=axs1.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        
                                        axs1.spines['top'].set_linewidth(2)
                                        axs1.spines['right'].set_linewidth(2)
                                        axs1.spines['bottom'].set_linewidth(2)
                                        axs1.spines['left'].set_linewidth(2)
                                        axs1.xaxis.set_tick_params(width=2)
                                        axs1.yaxis.set_tick_params(width=2)  
    
                                        # ----------------------------------------------------------------------------------------------------------          
                                        # STA/LTA
                                        # ----------------------------------------------------------------------------------------------------------
                                                
                                        st_cft = st_full.copy()
                
                                        for tr in st_cft:
                                            tr.detrend("linear")
                                            tr.taper(max_percentage=0.05, type='cosine')
                                            tr.filter('lowpass', freq=FREQ_CFT, corners=4, zerophase=True)
        
                                        cft = classic_sta_lta_py(st_full[0].data, int((STA_short/2)*st_full[0].stats.sampling_rate), int((LTA_short/2)*st_full[0].stats.sampling_rate))
                                        
                                        # STA/LTA max plot
    
                                        sta_lta_max_amp = np.max(cft)
                                        sta_lta_max_time = time_x_axis[np.argmax(cft)]
                                        axs2.scatter(sta_lta_max_time,sta_lta_max_amp, edgecolors='r',facecolors='None',marker='o',linewidths=1)
                                        
                                        axs2.text(x=0.5,y=0.9,s='max: '+str(round(sta_lta_max_amp,1)),ha="center", va="center", transform=axs2.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        axs2.text(x=0.99,y=0.95,s='THRON: '+str(THRON)+'\n'+'THROFF: '+str(THROFF),ha="right", va="top",transform=axs2.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
                                        axs2.text(x=0.01,y=0.03,s='STA: '+str(STA_short)+'\n'+'LTA: '+str(LTA_short),ha="left", va="bottom",transform=axs2.transAxes,bbox=dict(boxstyle="square",ec='k',fc='w'))
    
                                        line_full, = axs2.plot(time_x_axis,cft, 'k',label=str(freqs[0])+'-'+str(freqs[1])+' Hz')
                                        axs2.set_ylim(0, 10)
                                        axs2.legend(handles=[line_full],loc='upper left', fontsize=10, edgecolor='k', labelcolor='k')  
                                        
                                        axs2.axhline(xmin=0,xmax=1, y=THRON, color='darkorange',lw=0.5,linestyle='--')
                                        axs2.axhline(xmin=0,xmax=1, y=THROFF, color='tan',lw=0.5,linestyle='--')
    
                                        axs2.axvline(x=0,ymin=0, ymax=1, color='k',lw=0.5,linestyle='--')
                                        axs2.set_xlim(-event_cut_sta_lta_band,event_cut_sta_lta_band)
                                        axs2.xaxis.set_major_locator(mins2)
                                        axs2.xaxis.set_minor_locator(mins1)

                                        axs2.spines['top'].set_linewidth(2)
                                        axs2.spines['right'].set_linewidth(2)
                                        axs2.spines['bottom'].set_linewidth(2)
                                        axs2.spines['left'].set_linewidth(2)
                                        axs2.xaxis.set_tick_params(width=2)
                                        axs2.yaxis.set_tick_params(width=2)  
    
                                        if sta_lta_max_amp >= THRON and snr > SNR_MIN:
                                            check_SNR_STA_LTA.append(True)
                                        else:
                                            check_SNR_STA_LTA.append(False)
                                
                                    srn_per_band_lst.append(snr)                             
                                    sta_lta_max_per_band_lst.append(sta_lta_max_amp)                            
                                    noise_per_band_lst.append(nivel_ruido_db)
        
                                # ----------------------------------------------------------------------------------------------------------
                                # Saving figure
                                # ----------------------------------------------------------------------------------------------------------
                                
                                os.makedirs(FOLDER_OUTPUT+'FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION/'+model_earth+'/'+P_label+'/',exist_ok=True)
                                fig.savefig(FOLDER_OUTPUT+'FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION/'+model_earth+'/'+P_label+'/event_mseed_detection_'+model_earth+'_'+P_label+'_'+ev_time.strftime('%Y_%m_%d_%H_%M_%S')+'_'+name_glider+'_'+ev.event_descriptions[0].text+'.png',pad_inches=0.02,dpi=100)
                                plt.close()
                         
                                # ----------------------------------------------------------------------------------------------------------
                                # Saving figure selected
                                # ----------------------------------------------------------------------------------------------------------                          
                                
                                if condition(check_SNR_STA_LTA):
                                    os.makedirs(FOLDER_OUTPUT+'FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED/'+model_earth+'/'+P_label+'/',exist_ok=True)
                                    fig.savefig(FOLDER_OUTPUT+'FIGURAS/EVENT_MSEED_TELESEISMIC_DETECTION_SELECTED/'+model_earth+'/'+P_label+'/event_mseed_detection_'+model_earth+'_'+P_label+'_'+ev_time.strftime('%Y_%m_%d_%H_%M_%S')+'_'+name_glider+'_'+ev.event_descriptions[0].text+'.png',pad_inches=0.02,dpi=100)
                                    plt.close()
    
                               
                                # ----------------------------------------------------------------------------------------------------------
                                # Saving dataframe
                                # ----------------------------------------------------------------------------------------------------------
                                df = pd.DataFrame([[ev_file_mseed],[name_glider],[P_label],[ev.event_descriptions[0].text],[ev_time],[ev_lat],[ev_long],[ev_depth],[gcarc],[ev.magnitudes[0].mag],[ev.magnitudes[0].magnitude_type],[mt],[event],[time_glider],[lat_glider],[lon_glider],[dep_glider],[time_event_info],[lat_event_info],[lon_event_info],[depth_event_info],[srn_per_band_lst],[sta_lta_max_per_band_lst],[noise_per_band_lst]], index=['filename_mseed','name_glider','Phase','ev_ID','ev_time','ev_lat','ev_long','ev_depth','distance','magnitude','magnitude_type','moment_tensor','phases_time','time_glider','lat_glider','lon_glider','dep_glider','time_dive_info','lat_dive_info','lon_dive_info','depth_dive_info','srn_per_band','sta_lta_max_per_band','noise_per_band']).T
    
                                return df

In [ ]:
fases_sismicas = ["P","PKP","PKIKP"]
input_data = []
for fs in fases_sismicas:
    for ev in cat:
        input_data.append([ev,fs])

In [ ]:
df_results = []
with Pool(processes=20) as p:
    max_ = len(input_data)
    with tqdm(total=max_) as pbar:
        for result in p.imap_unordered(event_search,input_data):
            df_results.append(result)           
            pbar.update()

In [ ]:
dataframe_events_glider = pd.concat(df_results, ignore_index=True)

In [ ]:
def round_up_to_base(x, base=25):
    return x + (base - x) % base

In [ ]:
dataframe_events_glider['year'] =[UTCDateTime(i).year for i in dataframe_events_glider['ev_time']]
dataframe_events_glider['ev_depth_round'] = dataframe_events_glider['ev_depth'].apply(round_up_to_base)

# FILTRANDO O CATÀLOGO INICIAL EM FUNÇÃO DOS EVENTOS QUE ESTAVAM DENTRO DA JANELA DE DADOS

In [ ]:
filtered_catalog = []
for i in dataframe_events_glider.ev_ID.unique():
    for j in cat:
        if i == j.event_descriptions[0].text:
            filtered_catalog.append(j)

In [ ]:
len(filtered_catalog)

In [ ]:
# Obtaining all the parameters from the moment tensor via FMC.py
results_lst = []
for evet in filtered_catalog:
    Exponent_ = 1
    results_lst.append(str(evet.origins[0].longitude)+' '+str(evet.origins[0].latitude)+' '+str(evet.origins[0].depth/1000)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_rr)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_tt)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_pp)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_rt)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_rp)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_tp)+' '
      +str(Exponent_)+' '+' X Y '+evet.event_descriptions[0].text)

In [ ]:
file = open('earthquakes_glider.dat','w')
for item in results_lst:
	file.write(item+"\n")
file.close()

In [ ]:
os.system('python FMC.py -p '+FOLDER_OUTPUT+'FIGURAS/STATISTICS/Glider_data.pdf earthquakes_glider.dat')

In [ ]:
fig = plt.figure(figsize=(10,20))
mpl.rcParams.update({'font.size': 12})

#####################################################
# Adding MAP 
#####################################################

ax = fig.add_subplot(4,1,1,projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90])

# Normalizar valores para o intervalo [0, 1]
min_val = min(dataframe_events_glider.ev_depth_round.to_list())
max_val = max(dataframe_events_glider.ev_depth_round.to_list())
normalized_values = [(x - min_val) / (max_val - min_val) for x in dataframe_events_glider.ev_depth_round.to_list()]

# Obter as cores do colormap 'Plasma'
colors = [plt.cm.Spectral(value) for value in normalized_values]

# Converter as cores RGBA para formato hexadecimal (opcional)
hex_colors = [mcolors.rgb2hex(color) for color in colors]

for i in range(len(dataframe_events_glider['moment_tensor'])):
    bb = beach(dataframe_events_glider['moment_tensor'].values[i], xy=(dataframe_events_glider['ev_long'].values[i], dataframe_events_glider['ev_lat'].values[i]),facecolor=hex_colors[i],width=dataframe_events_glider['magnitude'].values[i]*1.5, linewidth=1)
    bb.set_zorder(10)
    ax.add_collection(bb)

ax.scatter(dataframe_events_glider['lon_glider'],dataframe_events_glider['lat_glider'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_events_glider['time_glider']]),marker='<',edgecolor='k',alpha=0.5,cmap='plasma',s=2.5,transform=proj)

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='k', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.left_ylabels = False
gl.xlabel_style = {'size': 12, 'color': 'k'}
gl.ylabel_style = {'size': 12, 'color': 'k'}
gl.xlabel_style = {'color': 'k', 'weight': 'bold'}
gl.ylabel_style = {'color': 'k', 'weight': 'bold'}
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.BORDERS, linestyle='--',lw=0.1)
ax.add_feature(cfeature.RIVERS)
ax.set_title(str(len(filtered_catalog))+'/'+str(len(cat))+' teleseismic earthquakes (Potential: '+str(round(len(filtered_catalog)/len(cat)*100))+'%)',fontsize=15,y=1.1)
ax.text(x=0.05,y=0.9,s='a)',ha="center", va="center",transform=ax.transAxes,fontsize=15)

#####################################################
# Adding inset 
#####################################################

axins = plt.axes([0.45, 0.69, 0.15, 0.15],projection=proj)

h = axins.scatter(dataframe_events_glider['lon_glider'],dataframe_events_glider['lat_glider'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_events_glider['time_glider']]),marker='<',edgecolor='k',alpha=0.5,cmap='plasma',s=50,transform=proj)
axins.add_feature(cfeature.LAND)
axins.add_feature(cfeature.OCEAN)
axins.add_feature(cfeature.COASTLINE,linewidth=0.3)
axins.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)

# subregion of the original image
x1 = np.min(dataframe_glider_xml['longitude'])-abs(np.min(dataframe_glider_xml['longitude'])/30)
x2 = np.max(dataframe_glider_xml['longitude'])+abs(np.max(dataframe_glider_xml['longitude'])/30)
y1 = np.min(dataframe_glider_xml['latitude'])-abs(np.min(dataframe_glider_xml['latitude'])/30)
y2 = np.max(dataframe_glider_xml['latitude'])+abs(np.max(dataframe_glider_xml['latitude'])/30)
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# ----------------------------------------------------------------------------------------------------------
# Adding grid 
gl = axins.gridlines(crs=ccrs.PlateCarree(),draw_labels=["bottom", "left", "right"],linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
axins.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=True, labeltop=False, labelleft=True, labelright=True)
gl.xlocator = FixedLocator([int(i) for i in np.linspace(x1,x2,num=4)])
gl.ylocator = FixedLocator([int(i) for i in np.linspace(y1,y2,num=4)])
gl.xformatter = LongitudeFormatter(direction_label=False)
gl.yformatter = LatitudeFormatter(direction_label=False)

#####################################################
# Adding inset DEPTH
#####################################################

ax_histx = plt.axes([0.45, 0.805, 0.15, 0.025],sharex=axins,facecolor='lightsteelblue')
ax_histx.scatter(dataframe_events_glider['lon_glider'],dataframe_events_glider['dep_glider'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_events_glider['time_glider']]),marker='<',edgecolor='k',alpha=0.5,cmap='plasma',s=50)
ax_histx.grid(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax_histx.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=False, labeltop=False, labelleft=True, labelright=True)
ax_histx.set_ylim(-1000,0)
ax_histx.set_xlim(x1, x2)
ax_histx.set_xticks([int(i) for i in np.linspace(x1,x2,num=4)])
ax_histx.yaxis.set_major_formatter('{x} m')
ax_histx.yaxis.set_major_locator(MultipleLocator(500))
ax_histx.yaxis.set_minor_locator(MultipleLocator(100))
ax_histx.patch.set_alpha(0.5)

# ----------------------------------------------------------------------------------------------------------
# Adding colorbar
divider = make_axes_locatable(axins)
ax_cb = divider.append_axes("bottom",size="4%", pad=0.25, axes_class=plt.Axes)

fig.add_axes(ax_cb)
cb = plt.colorbar(h, cax=ax_cb,orientation="horizontal")
cb.ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))

ax.indicate_inset_zoom(axins, edgecolor="black")

#####################################################
# Adding SCATTER PLOT 
#####################################################

ax1 = fig.add_subplot(4, 1, 2)

P_df = dataframe_events_glider[dataframe_events_glider.Phase == "P"]
PKP_df = dataframe_events_glider[dataframe_events_glider.Phase == "PKP"]
PKIKP_df = dataframe_events_glider[dataframe_events_glider.Phase == "PKIKP"]

sc1 = ax1.scatter(P_df.distance,P_df.magnitude,marker='o',c=P_df.ev_depth_round.to_list(),vmax=dataframe_events_glider.ev_depth_round.max(),vmin=dataframe_events_glider.ev_depth_round.min(),s=100,cmap='Spectral',edgecolors='k',alpha=0.5)
sc2 = ax1.scatter(PKP_df.distance,PKP_df.magnitude,marker='s',c=PKP_df.ev_depth_round.to_list(),vmax=dataframe_events_glider.ev_depth_round.max(),vmin=dataframe_events_glider.ev_depth_round.min(),s=100,cmap='Spectral',edgecolors='k',alpha=0.5)
sc3 = ax1.scatter(PKIKP_df.distance,PKIKP_df.magnitude,marker='^',c=PKIKP_df.ev_depth_round.to_list(),vmax=dataframe_events_glider.ev_depth_round.max(),vmin=dataframe_events_glider.ev_depth_round.min(),s=100,cmap='Spectral',edgecolors='k',alpha=0.5)

sc = [sc1,sc2,sc3]
ax1.legend(sc,["$P$","PKP","PKIKP"], loc='lower left',scatterpoints=1)

ax1.set_xlim(0,180)
ax1.set_ylim(5,8)
ax1.set_xlabel('Distance (deg)')
ax1.set_ylabel('Magnitude (mw)')
ax1.tick_params(axis='both',labelbottom=True, labeltop=True, labelleft=True, labelright=True,bottom=True, top=True, left=True, right=True)
ax1.text(x=0.05,y=0.9,s='b)',ha="center", va="center",transform=ax1.transAxes,fontsize=15)

#####################################################
# Adding HISTOGRAM PLOT 
#####################################################

ax2 = fig.add_subplot(4, 1, 3)
sc4 = pd.crosstab(dataframe_events_glider.Phase, dataframe_events_glider.ev_depth_round).plot.bar(stacked=False,cmap='Spectral',alpha=0.8,ax=ax2,legend=False)
ax2.set_xlabel('Seismic Phase')
ax2.set_ylabel('Frequency')
ax2.tick_params(axis='x', rotation=0, labelbottom=True, labeltop=False, labelleft=True, labelright=True,bottom=True, top=False, left=True, right=True)
ax2.tick_params(axis='y', labelbottom=True, labeltop=False, labelleft=True, labelright=True,bottom=True, top=False, left=True, right=True)
ax2.text(x=0.05,y=0.9,s='c)',ha="center", va="center",transform=ax2.transAxes,fontsize=15)

#####################################################
# Adding FOCAL MECHANISM PLOT 
#####################################################

from matplotlib.figure import Figure
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
ax3 = fig.add_subplot(4, 1, 4)
arr_img = plt.imread(FOLDER_OUTPUT+'FIGURAS/STATISTICS/glider_earthquakes_focal_mechanisms_potential.png')
ax3.imshow(arr_img)
ax3.axis('off')
ax3.text(x=0.05,y=0.9,s='d)',ha="center", va="center",transform=ax3.transAxes,fontsize=15)

# ----------------------------------------------------------------------------------------------------------
# Adding colorbar
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(sc2, cax=cbar_ax,label='Event Depth',fraction=0.025, shrink= 0.025)

# -------------------------------------------------------------------------------------------------------------------------------------------
os.makedirs(FOLDER_OUTPUT+'FIGURAS/STATISTICS/',exist_ok=True)
fig.savefig(FOLDER_OUTPUT+'FIGURAS/STATISTICS/database_earthquake_potential.png',pad_inches=0.025)


# Filtrando o banco de dados em função da análise visual dos resultados:

In [ ]:
selected_events = glob.glob(FOLDER_OUTPUT+'FIGURAS/EVENT_SELECTED_TELESEISMIC/*/*')

In [ ]:
selected_events_lst = []
selected_events_lst_phase = [] 
selected_events_lst_all = [] 
for img in selected_events:
    selected_events_lst.append(img.split('/')[-1].split('.')[0].split('_')[-1])
    selected_events_lst_phase.append(img.split('/')[-1].split('.')[0].split('_')[3])
    selected_events_lst_all.append([img.split('/')[-1].split('.')[0].split('_')[3],img.split('/')[-1].split('.')[0].split('_')[-1]])

In [ ]:
selected_events_lst_phase

In [ ]:
selected_events_lst = list(set(selected_events_lst))

In [ ]:
len(selected_events_lst)

In [ ]:
final_catalog = []
for i in selected_events_lst:
    for j in filtered_catalog:
        if i == j.event_descriptions[0].text:
            final_catalog.append(j)

In [ ]:
# Obtaining all the parameters from the moment tensor via FMC.py
results_final_lst = []
for evet in final_catalog:
    Exponent_ = 1
    results_final_lst.append(str(evet.origins[0].longitude)+' '+str(evet.origins[0].latitude)+' '+str(evet.origins[0].depth/1000)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_rr)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_tt)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_pp)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_rt)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_rp)+' '
      +str(evet.focal_mechanisms[0].moment_tensor.tensor.m_tp)+' '
      +str(Exponent_)+' '+' X Y '+evet.event_descriptions[0].text)

In [ ]:
file = open('earthquakes_glider_final.dat','w')
for item in results_final_lst:
	file.write(item+"\n")
file.close()

In [ ]:
os.system('python FMC.py -p '+FOLDER_OUTPUT+'FIGURAS/STATISTICS/Glider_final_data.pdf earthquakes_glider_final.dat')

In [ ]:
df_sel = []
for i,j in enumerate(selected_events_lst_all):
    dataframe_events_glider_final = dataframe_events_glider[(dataframe_events_glider['ev_ID'] == j[1]) & (dataframe_events_glider['Phase'] == j[0])]
    df_sel.append(dataframe_events_glider_final)

In [ ]:
dataframe_events_glider_final = dataframe_events_glider = pd.concat(df_sel, ignore_index=True)

In [ ]:
dataframe_events_glider_final.sort_values('distance')

In [ ]:
dataframe_events_glider_final.shape

In [ ]:
dataframe_events_glider_final.to_csv(FOLDER_OUTPUT+'FIGURAS/STATISTICS/table_events_selected.csv')

In [ ]:
dataframe_events_glider_final[dataframe_events_glider_final['Phase'] == 'P']

# FIGURA FINAL com terremotos, análises estatísticas e análise do mecanismo focal

In [ ]:
event_local = obspy.read_events(QUAKEXML_FOLDER+'event_2020_03_25_brazil.xml')

In [ ]:
fig = plt.figure(figsize=(20,10))
# Definindo globalmente o tamanho da fonte
mpl.rcParams.update({'font.size': 10})

#####################################################
# Adding MAP 
#####################################################

ax = fig.add_subplot(1,1,1,projection=ccrs.PlateCarree())

ax.set_extent([-180, 180, -90, 90])


# Normalizar valores para o intervalo [0, 1]
min_val = min(dataframe_events_glider.ev_depth_round.to_list())
max_val = max(dataframe_events_glider.ev_depth_round.to_list())
normalized_values = [(x - min_val) / (max_val - min_val) for x in dataframe_events_glider.ev_depth_round.to_list()]

# Obter as cores do colormap 'Plasma'
colors = [plt.cm.Spectral(value) for value in dataframe_events_glider.ev_depth_round.to_list()]

# Converter as cores RGBA para formato hexadecimal (opcional)
hex_colors = [mcolors.rgb2hex(color) for color in colors]

for i in range(len(dataframe_events_glider['moment_tensor'])):
    bb = beach(dataframe_events_glider['moment_tensor'].values[i], xy=(dataframe_events_glider['ev_long'].values[i], dataframe_events_glider['ev_lat'].values[i]),facecolor=hex_colors[i],width=dataframe_events_glider['magnitude'].values[i],alpha=0.2, linewidth=1)
    bb.set_zorder(10)
    ax.add_collection(bb)

# Normalizar valores para o intervalo [0, 1]
min_val = min(dataframe_events_glider_final.ev_depth_round.to_list())
max_val = max(dataframe_events_glider_final.ev_depth_round.to_list())
normalized_values = [(x - min_val) / (max_val - min_val) for x in dataframe_events_glider_final.ev_depth_round.to_list()]

# Obter as cores do colormap 'Plasma'
colors = [plt.cm.Spectral(value) for value in normalized_values]

# Converter as cores RGBA para formato hexadecimal (opcional)
hex_colors = [mcolors.rgb2hex(color) for color in colors]

for i in range(len(dataframe_events_glider_final['moment_tensor'])):
    bb = beach(dataframe_events_glider_final['moment_tensor'].values[i], xy=(dataframe_events_glider_final['ev_long'].values[i], dataframe_events_glider_final['ev_lat'].values[i]),facecolor=hex_colors[i],width=dataframe_events_glider_final['magnitude'].values[i]*1.5, linewidth=1)
    bb.set_zorder(10)
    ax.add_collection(bb)

ax.scatter(dataframe_events_glider_final['lon_glider'],dataframe_events_glider_final['lat_glider'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_events_glider_final['time_glider']]),marker='<',edgecolor='k',alpha=0.5,cmap='plasma',s=2.5,transform=proj)
ax.scatter(event_local[0].origins[-1].longitude,event_local[0].origins[-1].latitude,c="y",marker='*',edgecolor='k',alpha=0.9,s=100,transform=proj,label='Local event')

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=0.5, color='k', alpha=0.5, linestyle='--')
gl.top_xlabels = False
gl.left_ylabels = False
gl.xlabel_style = {'size': 12, 'color': 'k'}
gl.ylabel_style = {'size': 12, 'color': 'k'}
gl.xlabel_style = {'color': 'k', 'weight': 'bold'}
gl.ylabel_style = {'color': 'k', 'weight': 'bold'}
ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.BORDERS, linestyle='--',lw=0.1)
ax.add_feature(cfeature.RIVERS)
ax.set_title(str(len(selected_events_lst))+'/'+str(len(filtered_catalog))+' teleseismic earthquakes (Recovery: '+str(round(len(selected_events_lst)/len(filtered_catalog)*100))+'%)',fontsize=15,y=0.955)


ax.legend(loc='lower left')
#####################################################
# Adding inset GLIDER LOC
#####################################################

axins = plt.axes([0.45, 0.3, 0.20, 0.20],projection=proj)

h = axins.scatter(dataframe_events_glider_final['lon_glider'],dataframe_events_glider_final['lat_glider'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_events_glider_final['time_glider']]),marker='<',edgecolor='k',alpha=0.5,cmap='plasma',s=50,transform=proj)
axins.scatter(event_local[0].origins[-1].longitude,event_local[0].origins[-1].latitude,c="y",marker='*',edgecolor='k',alpha=0.9,s=200,transform=proj)

axins.add_feature(cfeature.LAND)
axins.add_feature(cfeature.OCEAN)
axins.add_feature(cfeature.COASTLINE,linewidth=0.3)
axins.add_feature(cfeature.BORDERS, linestyle=':',linewidth=0.3)

# subregion of the original image
x1 = np.min(dataframe_glider_xml['longitude'])-abs(np.min(dataframe_glider_xml['longitude'])/30)
x2 = np.max(dataframe_glider_xml['longitude'])+abs(np.max(dataframe_glider_xml['longitude'])/30)
y1 = np.min(dataframe_glider_xml['latitude'])-abs(np.min(dataframe_glider_xml['latitude'])/30)
y2 = np.max(dataframe_glider_xml['latitude'])+abs(np.max(dataframe_glider_xml['latitude'])/30)
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# ----------------------------------------------------------------------------------------------------------
# Adding grid 
gl = axins.gridlines(crs=ccrs.PlateCarree(),draw_labels=["bottom", "left", "right"],linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
axins.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=True, labeltop=False, labelleft=True, labelright=True)
gl.xlocator = FixedLocator([int(i) for i in np.linspace(x1,x2,num=4)])
gl.ylocator = FixedLocator([int(i) for i in np.linspace(y1,y2,num=4)])
gl.xformatter = LongitudeFormatter(direction_label=False)
gl.yformatter = LatitudeFormatter(direction_label=False)

#####################################################
# Adding inset GLIDER DEPTH
#####################################################

ax_histx = plt.axes([0.49, 0.52, 0.12, 0.05],sharex=axins,facecolor='lightsteelblue')
ax_histx.scatter(dataframe_events_glider_final['lon_glider'],dataframe_events_glider_final['dep_glider'],c=np.array([mdates.date2num(obspy.UTCDateTime(i).datetime) for i in dataframe_events_glider_final['time_glider']]),marker='<',edgecolor='k',alpha=0.5,cmap='plasma',s=50)
ax_histx.scatter(event_local[0].origins[-1].longitude,-50,c="y",marker='*',edgecolor='k',alpha=0.9,s=100)

ax_histx.grid(linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
ax_histx.tick_params(axis='both',which="both",bottom=True, top=True, left=True, right=True,labelbottom=False, labeltop=False, labelleft=True, labelright=True)
ax_histx.set_ylim(-1000,0)
ax_histx.set_xlim(x1, x2)
ax_histx.set_xticks([int(i) for i in np.linspace(x1,x2,num=4)])
ax_histx.yaxis.set_major_formatter('{x} m')
ax_histx.yaxis.set_major_locator(MultipleLocator(500))
ax_histx.yaxis.set_minor_locator(MultipleLocator(100))
ax_histx.patch.set_alpha(0.5)

# ----------------------------------------------------------------------------------------------------------
# Adding colorbar
divider = make_axes_locatable(axins)
ax_cb = divider.append_axes("bottom",size="4%", pad=0.25, axes_class=plt.Axes)

fig.add_axes(ax_cb)
cb = plt.colorbar(h, cax=ax_cb,orientation="horizontal")
cb.ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))

ax.indicate_inset_zoom(axins, edgecolor="black")

#####################################################
# Adding SCATTER PLOT 
#####################################################

ax1 = plt.axes([0.155, 0.65, 0.175, 0.175])

P_df = dataframe_events_glider_final[dataframe_events_glider_final.Phase == 'P']
PKP_df = dataframe_events_glider_final[dataframe_events_glider_final.Phase == 'PKP']
PKIKP_df = dataframe_events_glider_final[dataframe_events_glider_final.Phase == 'PKIKP']

sc1 = ax1.scatter(P_df.distance,P_df.magnitude,marker='o',c=P_df.ev_depth_round.to_list(),vmax=dataframe_events_glider.ev_depth_round.max(),vmin=dataframe_events_glider.ev_depth_round.min(),s=100,cmap='Spectral',edgecolors='k',alpha=0.5)
sc2 = ax1.scatter(PKP_df.distance,PKP_df.magnitude,marker='s',c=PKP_df.ev_depth_round.to_list(),vmax=dataframe_events_glider.ev_depth_round.max(),vmin=dataframe_events_glider.ev_depth_round.min(),s=100,cmap='Spectral',edgecolors='k',alpha=0.5)
sc3 = ax1.scatter(PKIKP_df.distance,PKIKP_df.magnitude,marker='^',c=PKIKP_df.ev_depth_round.to_list(),vmax=dataframe_events_glider.ev_depth_round.max(),vmin=dataframe_events_glider.ev_depth_round.min(),s=100,cmap='Spectral',edgecolors='k',alpha=0.5)

sc = [sc1,sc2,sc3]
ax1.legend(sc,["$P$","PKP","PKIKP"], loc='lower left',scatterpoints=1,ncols=3)

ax1.set_xlim(0,180)
ax1.set_ylim(5,8)
ax1.set_xlabel('Distance (deg)')
ax1.set_ylabel('Magnitude (mw)')
ax1.tick_params(axis='both',labelbottom=True, labeltop=True, labelleft=True, labelright=True,bottom=True, top=True, left=True, right=True)
ax1.text(x=0.05,y=0.9,s='a)',ha="center", va="center",transform=ax1.transAxes,fontsize=15)
ax1.patch.set_alpha(0.75)

#####################################################
# Adding HISTOGRAM PLOT 
#####################################################

ax2 = plt.axes([0.155, 0.22, 0.175, 0.15])

sc4 = pd.crosstab(dataframe_events_glider_final.Phase, dataframe_events_glider_final.ev_depth_round).plot.bar(stacked=False,cmap='Spectral',alpha=0.75,ax=ax2,legend=False)
ax2.set_xlabel('Seismic Phase')
ax2.set_ylabel('Frequency')
ax2.tick_params(axis='x', rotation=0,labelbottom=True, labeltop=False, labelleft=True, labelright=True,bottom=True, top=False, left=True, right=True)
ax2.tick_params(axis='y', labelbottom=True, labeltop=False, labelleft=True, labelright=True,bottom=True, top=False, left=True, right=True)
ax2.text(x=0.05,y=0.9,s='b)',ha="center", va="center",transform=ax2.transAxes,fontsize=15)
ax2.patch.set_alpha(0.75)

#####################################################
# Adding FOCAL MECHANISM PLOT 
#####################################################

ax3 = plt.axes([0.681, 0.151, 0.25, 0.25])
arr_img = plt.imread(FOLDER_OUTPUT+'FIGURAS/STATISTICS/glider_earthquakes_focal_mechanisms.png')
ax3.imshow(arr_img)
ax3.axis('off')
ax3.text(x=0.05,y=0.9,s='c)',ha="center", va="center",transform=ax3.transAxes,fontsize=15)


# ----------------------------------------------------------------------------------------------------------
# Adding colorbar
cbar_ax = fig.add_axes([0.72, 0.82, 0.15, 0.01])
cbar = fig.colorbar(sc2, cax=cbar_ax,label='Event Depth',orientation='horizontal')
# -------------------------------------------------------------------------------------------------------------------------------------------
os.makedirs(FOLDER_OUTPUT+'FIGURAS/STATISTICS/',exist_ok=True)
fig.savefig(FOLDER_OUTPUT+'FIGURAS/STATISTICS/database_earthquake_recorded.png',pad_inches=0.0,dpi=200)
